In [ ]:
import sqlite3
import requests
import pandas as pd
from pathlib import Path
from utils import get_job_description

In [ ]:
CREATE_TABLE_QUERY = Path('../sql/tables.sql').read_text()

In [ ]:
def get_job_listing() -> dict:

    return requests.get(
        'https://porsche-beesite-production-gjb.app.beesite.de/search/?data='
        """{
        "LanguageCode": "EN",
        "SearchParameters": {
            "FirstItem": 1,
            "CountItem": 1000,
            "Sort": [
                {
                    "Criterion": "PublicationStartDate",
                    "Direction": "DESC"
                }
            ],
            "MatchedObjectDescriptor": [
                "ID",
                "PositionTitle",
                "PositionURI",
                "PositionShortURI",
                "PositionLocation.CountryName",
                "PositionLocation.CityName",
                "PositionLocation.Longitude",
                "PositionLocation.Latitude",
                "PositionLocation.PostalCode",
                "PositionLocation.StreetName",
                "PositionLocation.BuildingNumber",
                "PositionLocation.Distance",
                "JobCategory.Name",
                "PublicationStartDate",
                "ParentOrganizationName",
                "ParentOrganization",
                "OrganizationShortName",
                "CareerLevel.Name",
                "JobSector.Name",
                "PositionIndustry.Name",
                "PublicationCode",
                "PublicationChannel.Id"
            ]
        },
        "SearchCriteria": [
            {
                "CriterionName": "PublicationChannel.Code",
                "CriterionValue": [
                    "12"
                ]
            },
            {
                "CriterionName": "CareerLevel.Code",
                "CriterionValue": [
                    "5"
                ]
            }
        ]
    }"""
    ).json()['SearchResult']['SearchResultItems']

In [ ]:
jobs = list()
for item in get_job_listing():
    job = item['MatchedObjectDescriptor']

    link = str(job['PositionURI']).replace(' ', '')

    jobs.append({
        "job_portal_id":    int(job['ID']),
        "job_link":         link,
        "job_title":        job['PositionTitle'],
        "job_description":  get_job_description(link),
        "job_posting_date": pd.to_datetime(str(job['PublicationStartDate']), format = 'ISO8601').date(),
        "job_type":         job['JobCategory'][0]['Name'],
        "job_field":        job['PositionIndustry'][0]['Name'],
        "job_city":         job['PositionLocation'][0]['CityName'],
    })

jobs = pd.DataFrame(jobs)
display(jobs)

In [ ]:
FILTER_QUERY = """SELECT job_portal_id FROM jobs WHERE job_link LIKE 'https://jobs.porsche.com/%'"""

In [ ]:
with sqlite3.connect('../data/jobs.db') as connection:

    connection.executescript(CREATE_TABLE_QUERY)

    already_exists = pd.read_sql(FILTER_QUERY, connection)
    already_exists = already_exists['job_portal_id']

    jobs = jobs[~jobs['job_portal_id'].isin(already_exists)]

    jobs.to_sql(
        name = 'jobs',
        con = connection,
        if_exists = 'append',
        index = False,
    )